https://aveshverma.github.io/sitemap.xml

In [4]:
# !pip -q install langchain
# !pip -q install bitsandbytes accelerate transformers
# !pip -q install sentence_transformers

In [5]:
# !pip -q install unstructured
# !pip install pinecone-client==2.2.4

In [1]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
import pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import torch

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


### Extract Data from URL

In [2]:
URLs = [
    "https://aveshverma.github.io/"
]

In [3]:
loader = UnstructuredURLLoader(urls = URLs)
data = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


##### AttributeError: module 'numpy.linalg._umath_linalg' has no attribute'_ilp64'

In [4]:
#!pip install numpy==1.24.4

In [5]:
data

[Document(page_content="Avesh Verma\n\nMenu\n\nHome\n\nAbout\n\nResume\n\nCompetencies\n\nSkills\n\nProjects\n\nContact\n\nHello!\n\nI'm Avesh Verma\n\nYour Data Science Consultant\n\nConnect with me! My Projects\n\nAbout\n\nAbout Me\n\nSeasoned ML Engineer skilled in Python, NLP, MLOps, and Generative AI, with a knack for seamless project execution and team collaboration.\n\nName: Avesh Verma\n\nDate of birth: December 04, 1997\n\nLet's Connect! LinkedIn\n\nEmail: verma.avesh001@gmail.com\n\nPhone:  +91-7258018577\n\n0\n\t                Projects completed\n\nDownload CV\n\nResume\n\nResume\n\nMachine Learning Engineer with 4 years of experience, specializing in AI solutions. Proven expertise in end-to-end\n\t\t\t\tproject execution and collaboration with cross-functional teams. Skilled in Python, SQL, Natural Language\n\t\t\t\tProcessing, MLOps and Generative AI, I am committed to continuous learning in the evolving field of data science..\n\n2023-Present\n\nData Science Consultant\n

###  Chunking

In [6]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

In [7]:
text_chunks=text_splitter.split_documents(data)

In [8]:
len(text_chunks)

6

In [9]:
text_chunks[0]

Document(page_content="Avesh Verma\nMenu\nHome\nAbout\nResume\nCompetencies\nSkills\nProjects\nContact\nHello!\nI'm Avesh Verma\nYour Data Science Consultant\nConnect with me! My Projects\nAbout\nAbout Me\nSeasoned ML Engineer skilled in Python, NLP, MLOps, and Generative AI, with a knack for seamless project execution and team collaboration.\nName: Avesh Verma\nDate of birth: December 04, 1997\nLet's Connect! LinkedIn\nEmail: verma.avesh001@gmail.com\nPhone:  +91-7258018577\n0\n\t                Projects completed\nDownload CV\nResume\nResume\nMachine Learning Engineer with 4 years of experience, specializing in AI solutions. Proven expertise in end-to-end\n\t\t\t\tproject execution and collaboration with cross-functional teams. Skilled in Python, SQL, Natural Language\n\t\t\t\tProcessing, MLOps and Generative AI, I am committed to continuous learning in the evolving field of data science..\n2023-Present\nData Science Consultant\nInfogain, Noida", metadata={'source': 'https://aveshver

### Embedding

In [10]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
qr = embeddings.embed_query("Hi")
len(qr)

384

In [12]:
import os

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', 'Enter Pinecone API Key here')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [13]:
import pinecone
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "portfolio-chatbot" # put in the name of your pinecone index here


In [14]:
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

### Create LLM Wrapper

In [15]:
notebook_login()

In [16]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)


model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                              load_in_8bit=True,
                                              #load_in_4bit=True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [17]:
pipe = transformers.pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [18]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [19]:
llm.predict("Please provide a concise summary of the Book Game of Thrones")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Please provide a concise summary of the Book Game of Thrones by George R. R. Martin.\nThe book "A Game of Thrones" by George R. R. Martin is the first installment in the "A Song of Ice and Fire" series. It follows the perspectives of multiple characters in the fictional world of Westeros, where the ruling families are embroiled in a complex and bloody struggle for control of the Iron Throne. The book introduces the main players in the story, including the Stark, Lannister, and Targaryen families, and sets the stage for the political intrigue and violence that will unfold throughout the series.\n\nThe book is known for its complex characters, detailed world-building, and unexpected plot twists, which have made it a favorite among fans of fantasy literature. It has also been adapted into a popular television series, which has further increased its popularity and exposure.\n\nOverall, "A Game of Thrones" is a gripping and immersive read that sets the stage for the rest of the series, and

### Initializing the Retrieval QA

In [20]:
from langchain.chains import RetrievalQA

In [21]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [28]:
query = "Has Avesh worked in Deep Learning?"

In [30]:
res = qa.run(query)

In [37]:
def print_between_keywords(input_string, start_keyword, end_keyword):
    start_index = input_string.find(start_keyword)
    end_index = input_string.find(end_keyword)

    if start_index != -1 and end_index != -1:  # If both keywords are found
        start_index += len(start_keyword)  # Adjust start index to exclude the start keyword itself
        print(input_string[start_index:end_index])  # Print the substring between the two keywords
    else:
        print("Start or end keyword not found in the input string.")

# Example usage
start_keyword = "Helpful Answer: "
end_keyword = "Unhelpful"

print_between_keywords(res, start_keyword, end_keyword)


Yes, Avesh has worked in Deep Learning. In his portfolio, he has listed several projects that involve deep learning techniques such as CNNs, LSTMs, and GRUs. For example, he worked on a project for chest cancer classification using deep learning and DVC. Additionally, he has experience with popular deep learning frameworks such as TensorFlow and PyTorch.


